# Домашнее задание № 8

> Только перед самой сдачей я обнаружил, что не посмотрел как следует на семинарскую тетрадку и взял не тот датасет — я думал, имелся в виду датасет 40k, который мы использовали в предыдущих двух домашках, поэтому взял оттуда семпл в 25% (около 8к в трейне). Поэтому все эксперименты проводились не дольше трёх эпох, но на большем количестве данных! Простите :D

## Задание 1 (7 баллов).
Это задание основано на этой тетрадке - https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/transfer_learning_hg/Fine_tunining_pretrained_LMs_torch.ipynb

На датасете lenta_sample.ru  дообучите две модели - modernbert-base (из семинара) и rumodernbert-base (https://huggingface.co/deepvk/RuModernBERT-base). Оцените разницу в качестве сравнив поклассовые метрики (classification_report)

Для обоих моделей качество должно быть >0.10 по f-мере (прогоните несколько экспериментов если у вас получаются нули, изменяя параметры).
Также для обоих моделей попробуйте дообучать модель и целиком и дообучать только последний слой.
Для RuModernBERT дополнительно сравните модель, которая использует первый вектор (cls токен, как в семинаре), так и усредненный вектор по всем hidden_state, который выдает bert.


### Подзадание на 1 балл
Дообучите любую из моделей, добавляя к BERT основе не только один линейный слой, но и один слой LSTM. Финальное состояние из этого слоя должно подаваться с новый линейный классификационный слой.

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
from tqdm.notebook import tqdm


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


### 1. Загрузка и подготовка данных и разделение на выборки

Я взял 25% оригинального 40-тысячного датасета: при таком сетапе каждый эксперимент занимал полчаса-час в зависимости от количества обучающих параметрах (на трёх эпохах)

In [ ]:
url = "https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/lenta_40k.csv.zip"
data = pd.read_csv(url)

data.dropna(subset=['topic', 'text'], inplace=True)

# Убираем редкие классы
data = data[~data['topic'].isin(['Легпром', 'Библиотека'])]

print(f"Total records: {len(data)}")
print(data['topic'].value_counts())

Total records: 44347
topic
Россия               9622
Мир                  8193
Экономика            4768
Спорт                3894
Наука и техника      3204
Культура             3183
Бывший СССР          3182
Интернет и СМИ       2643
Из жизни             1679
Дом                  1315
Силовые структуры    1203
Ценности              460
Бизнес                433
Путешествия           418
69-я параллель         82
Крым                   43
Культпросвет           25
Name: count, dtype: int64


In [ ]:
SAMPLE_FRACTION = 0.25

if SAMPLE_FRACTION < 1.0:
    data = data.sample(frac=SAMPLE_FRACTION, random_state=42)
    print(f"Data reduced to: {len(data)} records")

# Кодируем лейблы
labels = data['topic'].unique()
id2label = {i: l for i, l in enumerate(labels)}
label2id = {l: i for i, l in id2label.items()}

# Делим на выборки
train_data, val_data = train_test_split(
    data, test_size=0.2, random_state=42, stratify=data['topic'])

print(f"Train size: {len(train_data)}, Val size: {len(val_data)}")

Data reduced to: 11087 records
Train size: 8869, Val size: 2218


### 2. Кастомный датасет с токенизацией

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

### 3. Цикл обучения и оценивания

In [ ]:
def train_epoch(model, data_loader, criterion, optimizer):
    model.train()
    total_loss = 0

    all_preds = []
    all_labels = []

    for batch in tqdm(data_loader, desc="Training", leave=False):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(data_loader)

    # Считаем метрики
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='weighted', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)

    return avg_loss, precision, recall, f1, acc


def eval_epoch(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(data_loader)

    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='weighted', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)

    return avg_loss, precision, recall, f1, acc, all_labels, all_preds

### 4. Функция для всех экспериментов

Общая для всех экспериментов, которая принимает модель, флаг для заморозки слоёв, и всякие другие штуки.

In [ ]:
def run_experiment(
        model_name,
        model_class,
        freeze_bert=False,
        epochs=3,
        batch_size=32,
        pooling_type='cls'):
    print(
        f"\n{
            '=' *
            20} Experiment: {model_name} (Freeze: {freeze_bert}, Pooling: {pooling_type}) {
            '=' *
            20}")

    # Загружаем токенизатор
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Делаем датасеты
    train_dataset = TextDataset(
        train_data['text'].values,
        [label2id[l] for l in train_data['topic']],
        tokenizer
    )
    val_dataset = TextDataset(
        val_data['text'].values,
        [label2id[l] for l in val_data['topic']],
        tokenizer
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Загружаем модель
    base_model = AutoModel.from_pretrained(model_name)

    # Если надо, морозим слои
    if freeze_bert:
        for param in base_model.parameters():
            param.requires_grad = False

    model = model_class(base_model, len(label2id)).to(device)

    # Оптимизатор и функция потерь
    optimizer = optim.AdamW(
        model.parameters(),
        lr=2e-4 if freeze_bert else 2e-5,
        weight_decay=1e-2)
    criterion = nn.CrossEntropyLoss()

    # Обучаем
    for epoch in tqdm(range(epochs), desc="Epochs"):
        train_loss, t_p, t_r, t_f1, t_acc = train_epoch(
            model, train_loader, criterion, optimizer)
        val_loss, v_p, v_r, v_f1, v_acc, _, _ = eval_epoch(
            model, val_loader, criterion)

        print(f"Epoch {epoch + 1}/{epochs}")
        print(
            f"Train - Loss: {train_loss:.4f} | Acc: {t_acc:.4f} | F1: {t_f1:.4f}")
        print(
            f"Val   - Loss: {val_loss:.4f} | Acc: {v_acc:.4f} | F1: {v_f1:.4f}")
        print("-" * 50)

    # Финальный отчет
    _, _, _, _, _, true_labels, pred_labels = eval_epoch(
        model, val_loader, criterion)
    print("\nClassification Report:")
    print(
        classification_report(
            true_labels,
            pred_labels,
            target_names=[
                id2label[i] for i in range(
                    len(label2id))]))

    # Чистим для следующих экспериментов
    del model, base_model, optimizer
    torch.cuda.empty_cache()

### 5. Архитектуры моделей

In [ ]:
# Стандартный классификатор с cls token
class BertClassifier(nn.Module):
    def __init__(self, bert, num_classes):
        super().__init__()
        self.bert = bert
        self.drop = nn.Dropout(0.3)
        self.fc = nn.Linear(bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        output = self.fc(self.drop(pooled_output))
        return output

In [ ]:
# Классификатор с использованием hidden_states
class BertMeanClassifier(nn.Module):
    def __init__(self, bert, num_classes):
        super().__init__()
        self.bert = bert
        self.drop = nn.Dropout(0.3)
        self.fc = nn.Linear(bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # Усредняем векторы скрытых состояний с учётом маски
        last_hidden_state = outputs.last_hidden_state
        input_mask_expanded = attention_mask.unsqueeze(
            -1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        mean_pooled = sum_embeddings / sum_mask

        output = self.fc(self.drop(mean_pooled))
        return output

In [ ]:
# BERT + LSTM
class BertLSTMClassifier(nn.Module):
    def __init__(self, bert, num_classes):
        super().__init__()
        self.bert = bert
        self.lstm = nn.LSTM(
            bert.config.hidden_size,
            bert.config.hidden_size,
            batch_first=True)
        self.drop = nn.Dropout(0.3)
        self.fc = nn.Linear(bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state

        # Прогоняем через LSTM
        _, (hn, _) = self.lstm(last_hidden_state)
        # Берём последнее скрытое состояние
        lstm_out = hn[-1]

        output = self.fc(self.drop(lstm_out))
        return output

### 6. Эксперимент 1: ModernBERT: дообучаем только голову классификации

In [ ]:
run_experiment(
    model_name="answerdotai/ModernBERT-base",
    model_class=BertClassifier,
    freeze_bert=True,
    epochs=3
)


==================== Experiment: answerdotai/ModernBERT-base (Freeze: True, Pooling: cls) ====================


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: answerdotai/ModernBERT-base
Key               | Status     |  | 
------------------+------------+--+-
head.norm.weight  | UNEXPECTED |  | 
head.dense.weight | UNEXPECTED |  | 
decoder.bias      | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Training:   0%|          | 0/278 [00:00<?, ?it/s]

...

Training:   0%|          | 0/278 [00:00<?, ?it/s]

Epoch 2/3
Train - Loss: 2.1766 | Acc: 0.2852 | F1: 0.2320
Val   - Loss: 2.0635 | Acc: 0.3431 | F1: 0.2730
--------------------------------------------------


Training:   0%|          | 0/278 [00:00<?, ?it/s]

Epoch 3/3
Train - Loss: 2.0835 | Acc: 0.3181 | F1: 0.2652
Val   - Loss: 1.9987 | Acc: 0.3485 | F1: 0.2785
--------------------------------------------------

Classification Report:
                   precision    recall  f1-score   support

            Спорт       0.55      0.18      0.27       201
  Наука и техника       0.55      0.32      0.40       158
              Мир       0.36      0.44      0.39       419
         Из жизни       0.00      0.00      0.00        90
         Культура       0.33      0.17      0.22       155
           Россия       0.31      0.86      0.46       475
   Интернет и СМИ       0.00      0.00      0.00       130
Силовые структуры       0.00      0.00      0.00        63
        Экономика       0.42      0.29      0.35       231
      Бывший СССР       0.00      0.00      0.00       161
           Бизнес       0.00      0.00      0.00        19
              Дом       1.00      0.01      0.03        69
             Крым       0.00      0.00      0.00   

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 7. Эксперимент 2: ModernBERT: полное дообучение

In [ ]:
run_experiment(
    model_name="answerdotai/ModernBERT-base",
    model_class=BertClassifier,
    freeze_bert=False,
    epochs=3,
    batch_size=16
)


==================== Experiment: answerdotai/ModernBERT-base (Freeze: False, Pooling: cls) ====================


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Training:   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 2/3
Train - Loss: 1.4385 | Acc: 0.5586 | F1: 0.5288
Val   - Loss: 1.2742 | Acc: 0.5996 | F1: 0.5705
--------------------------------------------------


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 3/3
Train - Loss: 1.0728 | Acc: 0.6688 | F1: 0.6547
Val   - Loss: 1.2345 | Acc: 0.6226 | F1: 0.6051
--------------------------------------------------

Classification Report:
                   precision    recall  f1-score   support

            Спорт       0.86      0.95      0.91       201
  Наука и техника       0.53      0.75      0.62       158
              Мир       0.61      0.77      0.68       419
         Из жизни       0.31      0.38      0.34        90
         Культура       0.64      0.72      0.68       155
           Россия       0.75      0.50      0.60       475
   Интернет и СМИ       0.64      0.26      0.37       130
Силовые структуры       0.57      0.06      0.11        63
        Экономика       0.61      0.73      0.67       231
      Бывший СССР       0.68      0.57      0.62       161
           Бизнес       0.00      0.00      0.00        19
              Дом       0.44      0.74      0.55        69
             Крым       0.00      0.00      0.00   

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 8. Эксперимент 3: RuModernBERT: дообучаем только голову классификации

In [ ]:
run_experiment(
    model_name="deepvk/RuModernBERT-base",
    model_class=BertClassifier,
    freeze_bert=True,
    epochs=3
)


==================== Experiment: deepvk/RuModernBERT-base (Freeze: True, Pooling: cls) ====================


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

...

Training:   0%|          | 0/278 [00:00<?, ?it/s]

Epoch 2/3
Train - Loss: 2.3264 | Acc: 0.2143 | F1: 0.1185
Val   - Loss: 2.3070 | Acc: 0.2538 | F1: 0.1397
--------------------------------------------------


Training:   0%|          | 0/278 [00:00<?, ?it/s]

Epoch 3/3
Train - Loss: 2.3081 | Acc: 0.2263 | F1: 0.1264
Val   - Loss: 2.2972 | Acc: 0.2326 | F1: 0.1108
--------------------------------------------------

Classification Report:
                   precision    recall  f1-score   support

            Спорт       0.00      0.00      0.00       201
  Наука и техника       0.00      0.00      0.00       158
              Мир       0.42      0.11      0.17       419
         Из жизни       0.00      0.00      0.00        90
         Культура       0.00      0.00      0.00       155
           Россия       0.22      0.99      0.36       475
   Интернет и СМИ       0.00      0.00      0.00       130
Силовые структуры       0.00      0.00      0.00        63
        Экономика       0.00      0.00      0.00       231
      Бывший СССР       0.00      0.00      0.00       161
           Бизнес       0.00      0.00      0.00        19
              Дом       0.00      0.00      0.00        69
             Крым       0.00      0.00      0.00   

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 9. Эксперимент 4: RuModernBERT: полное дообучение

In [ ]:
run_experiment(
    model_name="deepvk/RuModernBERT-base",
    model_class=BertClassifier,
    freeze_bert=False,
    epochs=3,
    batch_size=16
)


==================== Experiment: deepvk/RuModernBERT-base (Freeze: False, Pooling: cls) ====================


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: deepvk/RuModernBERT-base
Key               | Status     |  | 
------------------+------------+--+-
head.dense.weight | UNEXPECTED |  | 
decoder.bias      | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Training:   0%|          | 0/555 [00:00<?, ?it/s]

...

Training:   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 2/3
Train - Loss: 0.5687 | Acc: 0.8203 | F1: 0.8116
Val   - Loss: 0.6942 | Acc: 0.7696 | F1: 0.7681
--------------------------------------------------


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 3/3
Train - Loss: 0.3339 | Acc: 0.8945 | F1: 0.8919
Val   - Loss: 0.6922 | Acc: 0.7845 | F1: 0.7752
--------------------------------------------------

Classification Report:
                   precision    recall  f1-score   support

            Спорт       0.96      0.97      0.96       201
  Наука и техника       0.70      0.89      0.78       158
              Мир       0.81      0.79      0.80       419
         Из жизни       0.82      0.41      0.55        90
         Культура       0.78      0.89      0.83       155
           Россия       0.76      0.79      0.78       475
   Интернет и СМИ       0.82      0.65      0.72       130
Силовые структуры       0.58      0.24      0.34        63
        Экономика       0.75      0.85      0.80       231
      Бывший СССР       0.80      0.91      0.85       161
           Бизнес       0.14      0.05      0.08        19
              Дом       0.85      0.80      0.82        69
             Крым       0.00      0.00      0.00   

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 10. Эксперимент 5: RuModernBERT: полное дообучение с минпуллингом

In [ ]:
run_experiment(
    model_name="deepvk/RuModernBERT-base",
    model_class=BertMeanClassifier,
    freeze_bert=False,
    epochs=3,
    batch_size=16,
    pooling_type='mean'
)


==================== Experiment: deepvk/RuModernBERT-base (Freeze: False, Pooling: mean) ====================


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: deepvk/RuModernBERT-base
Key               | Status     |  | 
------------------+------------+--+-
head.dense.weight | UNEXPECTED |  | 
decoder.bias      | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Training:   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 1/3
Train - Loss: 0.9197 | Acc: 0.7222 | F1: 0.7099
Val   - Loss: 0.6827 | Acc: 0.7849 | F1: 0.7678
--------------------------------------------------


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 2/3
Train - Loss: 0.4914 | Acc: 0.8388 | F1: 0.8343
Val   - Loss: 0.6166 | Acc: 0.8003 | F1: 0.7939
--------------------------------------------------


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 3/3
Train - Loss: 0.2565 | Acc: 0.9126 | F1: 0.9114
Val   - Loss: 0.7024 | Acc: 0.7836 | F1: 0.7783
--------------------------------------------------


### 11. Эксперимент 6: RuModernBERT + LSTM

In [ ]:
run_experiment(
    model_name="deepvk/RuModernBERT-base",
    model_class=BertLSTMClassifier,
    freeze_bert=False,
    epochs=3,
    batch_size=16,
    pooling_type='lstm'
)


==================== Experiment: deepvk/RuModernBERT-base (Freeze: False, Pooling: lstm) ====================


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Training:   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 2/3
Train - Loss: 0.5558 | Acc: 0.8305 | F1: 0.8236
Val   - Loss: 0.7282 | Acc: 0.7728 | F1: 0.7653
--------------------------------------------------


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 3/3
Train - Loss: 0.3578 | Acc: 0.8886 | F1: 0.8854
Val   - Loss: 0.7467 | Acc: 0.7741 | F1: 0.7724
--------------------------------------------------

Classification Report:
                   precision    recall  f1-score   support

            Спорт       0.99      0.93      0.96       201
  Наука и техника       0.83      0.78      0.81       158
              Мир       0.79      0.68      0.73       419
         Из жизни       0.67      0.59      0.63        90
         Культура       0.83      0.88      0.86       155
           Россия       0.71      0.80      0.75       475
   Интернет и СМИ       0.72      0.68      0.70       130
Силовые структуры       0.67      0.67      0.67        63
        Экономика       0.78      0.85      0.81       231
      Бывший СССР       0.83      0.87      0.85       161
           Бизнес       0.18      0.16      0.17        19
              Дом       0.81      0.83      0.82        69
             Крым       0.00      0.00      0.00   

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 12. Выводы

Во-первых, сразу скажем очевидное: RuModernBERT значительно лучше при полном дообучении. Оно абсолютно ожидаемо, потому что оригинальный ModernBERT — строго англоязычная модель. Интереснее было бы сравнить мультиязычный ModernBERT, который вышел позже, с русскоязычным — вот тут результат мог бы быть неочевиден. Однако при дообучении только головы классификатора ситуация интереснее. В целом, обе модели справляются ужасно, но RuModernBERT сильно хуже оригинального ModernBERT'а. В целом, низкий F1-score ожидаем, потому что, как показали авторы Sentence-Transformers, ванильные BERT-like-модели ужасно производят эмбеддинги для предложений. Но проигрышь RuModernBERT по сравнению с оригиналом я тут объяснить затрудняюсь.

Во-вторых, конечно, полное дообучение (любой модели) даёт значительно лучший результат, чем обучение только головы классификатора. Стоит даже говорить о том, что обучение только головы классификатора здесь вообще не даёт хоть сколько-нибудь удовлетворительных результатов.

Что касается стратегии пуллинга (проверял только на RuModernBERT), особой разницы с точки зрения перформанса между ними нет. Класспуллинг и минпуллинг дают одинаковый F1, приделывание LSTM-слоя немного ухудшает результат, скорее всего, из-за переобучения в виду ненужной сложности, зато обучается чуточку быстрее.

Глядя на динамику обучения, можно отметить очень быструю сходимость для RuModernBERT (76% уже после первой эпохи), а вот оригинальный ModernBERT адаптировался к русскому тексту гораздо медленнее, что тоже не является откровением. Минпуллинг показывает самое явное переобучение (91% на трейне VS. 78% на тесте), у LSTM переобучение тоже сильное.

На редких классах RuModernBERT перформит тоже лучше, во всяком случае, полных нулей почти нет.

## Задание 2 (2 балла)

Проведите несколько экспериментов с gradual unfreezing. Используя ту же задачу и датасет, что в первом задании, дообучите модели таким образом, что в начале обучения весь backbone BERT заморожен и дообучение происходит только на новом Linear/dense/fc слое, но после каждой N эпохи, размораживайте (`_requires_grad = True`) какую-то часть параметров BERT. Проведите как минимум 3 эксперимента с разными методиками разморозки. Вы можете придумать их сами, но вот несколько примеров - 1) разморозка начинается с конца модели и каждые две эпохи размораживается несколько слоев до тех пор пока не разморозятся все слои всключая первый эмбеддинг слой 2) разморозка начинается с начала (то есть сначала размораживается только embedding слой), а со временем включаются и все последующие 3) таргетировано размораживаются сначала слои query трансформаций, потом key, потом value 4) или даже можете попробовать сначала разморозить все слои и наоборот замораживать части с прогрессом обучения. Для каждого эксперимента сохраните метрики на каждой эпохе и финальную метрику, сравните их между собой - какой schedule сработал лучше всего? Сработало ли что-то лучше дефолтного подхода из первого задания?

Будем использовать RuModernBERT, так как, как видно из первого задания, его резулььтаты гораздо лучше (оно и не то чтобы удивительно!)

### 1. Функции для управления слоями

Одна для определения количества слоёв в модели, другая — для управляемой заморозки слоёв.

In [ ]:
def get_num_layers(model):
    # Определяем количество слоёв
    layer_indices = set()
    for name, _ in model.named_parameters():
        parts = name.split('.')
        for p in parts:
            if p.isdigit():
                layer_indices.add(int(p))

    return max(layer_indices) + 1 if layer_indices else 0


def freeze_backbone(model):
    # Замораживаем все параметры
    for param in model.parameters():
        param.requires_grad = False

    # Размораживаем только голову классификатора (последний слой)
    for name, param in model.named_parameters():
        if 'fc' in name or 'classifier' in name:
            param.requires_grad = True

### 2. Функция для запуска экспериментов с разморозкой

Это не избавит нас писать функции для каждого эксперимента отдельно, как в первом задании, но мы сможем использовать эти функции как коллбеки.

In [ ]:
def run_gradual_experiment(schedule_name, unfreeze_callback, epochs=4):
    print(f"\n{'=' * 20} Experiment: {schedule_name} {'=' * 20}")

    model_name = "deepvk/RuModernBERT-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    base_model = AutoModel.from_pretrained(model_name)

    # Создаём классификационный слой
    model = BertClassifier(base_model, len(label2id)).to(device)

    # Морозим backbone полностью
    freeze_backbone(model)

    # Определяем кол-во слоёв
    num_layers = get_num_layers(base_model)
    print(f"Detected {num_layers} layers in the backbone.")

    optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
    criterion = nn.CrossEntropyLoss()

    train_dataset = TextDataset(
        train_data['text'].values, [
            label2id[l] for l in train_data['topic']], tokenizer)
    val_dataset = TextDataset(
        val_data['text'].values, [
            label2id[l] for l in val_data['topic']], tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16)

    history = []

    for epoch in range(epochs):
        # Коллбек для разморозки по расписанию
        unfreeze_callback(model, epoch, epochs, num_layers)

        # Считаем размороженные параметры
        trainable_params = sum(p.numel()
                               for p in model.parameters() if p.requires_grad)
        print(
            f"Epoch {
                epoch + 1}/{epochs}: {trainable_params} trainable parameters")

        # Обучаем
        train_loss, t_p, t_r, t_f1, t_acc = train_epoch(
            model, train_loader, criterion, optimizer)

        # Оцениваем
        val_loss, v_p, v_r, v_f1, v_acc, _, _ = eval_epoch(
            model, val_loader, criterion)

        print(f"Train F1: {t_f1:.4f} | Val F1: {v_f1:.4f}")
        history.append(v_f1)

    del model, base_model, optimizer
    torch.cuda.empty_cache()

    return history[-1]

### 3. Эксперимент 1: размораживаем с конца

Так как датасет (по моей вине) большой, и дольше трёх эпох обучаться проблематично, применим следующую логику: на первой эпохе разморозим нижнюю треть слоёв, на второй — середину, а на третьей — эмбеддинги.

In [ ]:
def schedule_end_to_start(model, epoch, total_epochs, num_layers):
    # Логика: делим слои на группы и размораживаем с конца
    # Первая эпоха: размораживаем верхнюю треть (ближе к выходу)
    # Вторая эпоха: середину
    # Третья+: эмбеддинги

    layers_per_step = num_layers // (total_epochs +
                                     1) if total_epochs > 1 else num_layers

    # Определяем индекс слоя, до которого мы размораживаем

    start_layer_index = max(0, num_layers - (epoch + 1) * layers_per_step)

    print(f"--> Unfreezing layers from {start_layer_index} to {num_layers}")

    for name, param in model.named_parameters():
        # Ищем цифру слоя в названии
        parts = name.split('.')
        layer_idx = -1
        for p in parts:
            if p.isdigit():
                layer_idx = int(p)
                break

        # Размораживаем, если индекс слоя >= текущего порога
        # Или если это не слой энкодера
        if layer_idx >= start_layer_index:
            param.requires_grad = True

    # Если это последняя эпоха, размораживаем вообще всё
    if epoch == total_epochs - 1:
        for param in model.parameters():
            param.requires_grad = True

In [ ]:
res_end_to_start = run_gradual_experiment(
    "Unfreeze End -> Start", schedule_end_to_start, epochs=3)


==================== Experiment: Unfreeze End -> Start ====================


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Train F1: 0.6450 | Val F1: 0.7390
--> Unfreezing layers from 0 to 22
Epoch 2/3: 110343185 trainable parameters


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Train F1: 0.7715 | Val F1: 0.7609
--> Unfreezing layers from 0 to 22
Epoch 3/3: 149027345 trainable parameters


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Train F1: 0.8350 | Val F1: 0.7822


### 4. Эксперимент 2: размораживаем с начала

Наоборот: сначала эмбеддинги, потом середину, а потом нижнюю треть.

In [ ]:
def schedule_start_to_end(model, epoch, total_epochs, num_layers):
    # Логика: размораживаем слои с начала

    layers_per_step = num_layers // (total_epochs -
                                     1) if total_epochs > 1 else num_layers

    end_layer_index = (epoch + 1) * layers_per_step

    print(f"--> Unfreezing layers from 0 to {end_layer_index}")

    for name, param in model.named_parameters():
        parts = name.split('.')
        layer_idx = -1
        for p in parts:
            if p.isdigit():
                layer_idx = int(p)
                break

        # Размораживаем, если слой попадает в диапазон [0, end_layer_index]
        if layer_idx != -1 and layer_idx < end_layer_index:
            param.requires_grad = True

        # Также размораживаем эмбеддинги на первом шаге
        if 'embedding' in name and epoch >= 0:
            param.requires_grad = True

In [ ]:
res_start_to_end = run_gradual_experiment(
    "Unfreeze Start -> End", schedule_start_to_end, epochs=3)


==================== Experiment: Unfreeze Start -> End ====================


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: deepvk/RuModernBERT-base
Key               | Status     |  | 
------------------+------------+--+-
head.dense.weight | UNEXPECTED |  | 
decoder.bias      | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Detected 22 layers in the backbone.
--> Unfreezing layers from 0 to 11
Epoch 1/3: 93861137 trainable parameters


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Train F1: 0.6070 | Val F1: 0.7338
--> Unfreezing layers from 0 to 22
Epoch 2/3: 149026577 trainable parameters


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Train F1: 0.8116 | Val F1: 0.7504
--> Unfreezing layers from 0 to 33
Epoch 3/3: 149026577 trainable parameters


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Train F1: 0.9001 | Val F1: 0.7591


> Тут надо заметить, что оно не получилось на 100% так, как я хотел, потому что слои эмбеддингов как бы отдельные и автоматизированно их индексить трудно, поэтому 2 и 3 эпохи по факту обучали всю модель

### 5. Эксперимент 3: таргетная разморозка

На первой эпохе размораживаем Query, на второй — Key, на третьей — Value.

In [ ]:
def schedule_qkv(model, epoch, total_epochs, num_layers):
    # Логика:
    # Первая эпоха: размораживаем только Query веса (во всех слоях)
    # Вторая: добавляем Key веса
    # Третья: добавляем Value веса + Dense

    target_substrings = []

    if epoch >= 0:
        print("--> Unfreezing QUERY weights")
        target_substrings.extend(['query', 'q_proj', 'Wq'])

    if epoch >= 1:
        print("--> Unfreezing KEY weights")
        target_substrings.extend(['key', 'k_proj', 'Wk'])

    if epoch >= 2:
        print("--> Unfreezing VALUE weights + Output")
        target_substrings.extend(['value', 'v_proj', 'Wv', 'dense', 'output'])

    for name, param in model.named_parameters():
        for substring in target_substrings:
            if substring in name:
                param.requires_grad = True

In [ ]:
res_qkv = run_gradual_experiment(
    "Targeted Q -> K -> V",
    schedule_qkv,
    epochs=3)


==================== Experiment: Targeted Q -> K -> V ====================


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: deepvk/RuModernBERT-base
Key               | Status     |  | 
------------------+------------+--+-
head.dense.weight | UNEXPECTED |  | 
decoder.bias      | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Detected 22 layers in the backbone.
--> Unfreezing QUERY weights
Epoch 1/3: 38941457 trainable parameters


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Train F1: 0.5553 | Val F1: 0.7450
--> Unfreezing QUERY weights
--> Unfreezing KEY weights
Epoch 2/3: 38941457 trainable parameters


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Train F1: 0.7930 | Val F1: 0.7731
--> Unfreezing QUERY weights
--> Unfreezing KEY weights
--> Unfreezing VALUE weights + Output
Epoch 3/3: 38941457 trainable parameters


Training:   0%|          | 0/555 [00:00<?, ?it/s]

Train F1: 0.8620 | Val F1: 0.7731


### 6. Красиво сравниваем

In [ ]:
print(f"\n{'=' * 20} Final Comparison {'=' * 20}")
print(f"1. End -> Start F1: {res_end_to_start:.4f}")
print(f"2. Start -> End F1: {res_start_to_end:.4f}")
print(f"3. Q -> K -> V  F1: {res_qkv:.4f}")

# Тупо по скору
best_score = max(res_end_to_start, res_start_to_end, res_qkv)
if best_score == res_end_to_start:
    print("Best strategy: Unfreezing from End to Start")
elif best_score == res_start_to_end:
    print("Best strategy: Unfreezing from Start to End")
else:
    print("Best strategy: Targeted QKV Unfreezing")


==================== Final Comparison ====================
1. End -> Start F1: 0.7822
2. Start -> End F1: 0.7591
3. Q -> K -> V  F1: 0.7731
Best strategy: Unfreezing from End to Start


### 7. Выводы.

Лучше всего себя показала стратегия, при которой мы начинали разморозку с конца и доходили до начала. Тут почти нет переобучения и самый высокий финальный скор. Отлично мэтчится с базовыми принципами transfer learning, которые говорят о том, что верхние слои адаптируются под task-specific признаки, а нижние — под общие языковые. Когда мы обучили сначала task-specific-слои, они хорошо адаптировались под нашу задачу, а более общие слои особо не сломались.

При разморозке от начала к концу произошло мощное переобучение (очень высокий F1 на трейне, но самый низкий на валидации). Тоже мэтчится с transfer learning: мы сначла разморозили эмбеддинги и испортили их тренировочными данными, модель их просто запомнила, а верхнеуровневая логика адаптировалась сильно меньше.

В третьем эксперименте модель после второй эпохи просто перестала обучаться, результат не улучшился. Очевидно из-за того, что мы заблокировали серединные слои.

Если сравнивать с экспериментами из первого задания, можно заметить, что gradual unfreezing (от конца к началу) сработал лучше, чем полное дообучение  напротяжении всех эпох. Итоговый F1 немного выше, и мы значительно уменьшили переобучение. По сути, ввели дополнительную регуляризацию: когда мы замораживаем в начале нижнеуровневые слои, мы уменьшаем эффект "катастрофического забывания" того, что модель уже знала в процессе претрейна.